# A little bit of polars select and pl_series_hash
note, I'm not using buckaroo here, because for simple static examples, buckaroo isn't the best fit

In [ ]:
import polars as pl
from pl_series_hash import hash_xx

In [ ]:
df = pl.DataFrame({'a':[10,20,30], 'b':[10.5, 20.5, 30.5], 'c':['foo','bar','baz']})
df

`pl.all()` says operate on all columns

`.hash()` is built into to polars, it applies an element-wise hash function, notice that the output shape is the same

In [ ]:
df.select(pl.all().hash())

`pl_series_hash` is the namespace created by my polars plugin

`hash_xx()` returns a single value

In [ ]:
df.select(pl.all().pl_series_hash.hash_xx())

# Multiple expressions

In [ ]:
import polars.selectors as cs
df.select([pl.all().pl_series_hash.hash_xx().name.prefix('hash_'),
           cs.numeric().mean().name.prefix("mean_")])

# LazyFrames
Polars LazyFrames are pointers to a datasource that queries can be executed on.  Crucially the entire dataframe isn't read into memory


In [ ]:
print(df.select([pl.all().len()]))
pl.scan_parquet("../citibike-trips-2016-04.parq").select(pl.all().len()).collect()

In [ ]:
# you can also slice
pl.scan_parquet("../citibike-trips-2016-04.parq")\
    .slice(0,5000)\
    .select(pl.all().len())\
    .collect()

# The bug in my polars plugin

In [1]:
!ls -alhstr ./PULocationID.parq

220680 -rw-r--r--@ 1 paddy  staff   108M Dec  8 11:13 ./PULocationID.parq


In [ ]:
from buckaroo.read_utils import read_df, read
import polars as pl
from pl_series_hash import hash_xx
ldf = read_df("../2024-01-05_tripdata.parq")
ldf

In [ ]:
%timeit ldf.select([pl.col('PULocationID')]).sink_parquet("PULocationID.parq")

In [ ]:
%timeit ldf.select([pl.col('PULocationID').pl_series_hash.hash_xx()]).collect()

In [ ]:
%timeit ldf.select([pl.col('PULocationID').mean()]).collect()

In [ ]:
%timeit ldf.select([pl.col('PULocationID').hash()]).collect()

In [ ]:
!time md5sum PULocationID.parq

In [ ]:
#108 MB
#1.52 seconds
# do the calcs